### Load data

In [ ]:
from glotaran.io import load_dataset

dataset = load_dataset("data/data.ascii")

### Analysis without equal area penalties

Example of a two step optimization
- First do a (partial) optimization with first scheme
- Extract the optimized schemed and adjust its settings
- Optimize again with the second scheme

In [ ]:
from glotaran.io import load_model, load_parameters
from glotaran.optimization.optimize import optimize
from glotaran.project.scheme import Scheme

model = load_model("models/model.yml")
parameters = load_parameters("models/parameters.yml")

scheme_no_penalties = Scheme(
    model=model,
    parameters=parameters,
    data={"dataset1": dataset},
    optimization_method="TrustRegionReflection",
    maximum_number_function_evaluations=5,
)

# optimization with first scheme
result = optimize(scheme_no_penalties)
# optimization with second scheme,
# where we change the maximum number of function evaluations
scheme2 = result.get_scheme()
scheme2.maximum_number_function_evaluations = 2
result_no_penalties = optimize(scheme2)

#### Saving results and creating plots

In [ ]:
from pyglotaran_extras.io import setup_case_study

from glotaran.io import save_result

base_results_folder, _ = setup_case_study(output_folder_name="pyglotaran_examples_results")
results_folder = base_results_folder / "no_penalties"
save_result(result_no_penalties, results_folder / "result.yml", allow_overwrite=True)

In [ ]:
from pyglotaran_extras.plotting.plot_overview import plot_simple_overview

plot_simple_overview(result_no_penalties.data["dataset1"]);

### Analysis using equal area penalties

This time we use equal parameter in our model, allowing us to estimate the relative amplitude between the two species.

In [ ]:
from glotaran.io import load_model, load_parameters
from glotaran.optimization.optimize import optimize
from glotaran.project.scheme import Scheme

parameters = load_parameters("models/parameters_equal_area_penalties.yml")
model = load_model("models/model_equal_area_penalties.yml")

scheme_equal_area = Scheme(
    model=model,
    parameters=parameters,
    data={"dataset1": dataset},
    optimization_method="TrustRegionReflection",
    maximum_number_function_evaluations=7,
)

result_equal_area = optimize(scheme_equal_area)

#### Plotting

In [ ]:
from pyglotaran_extras.plotting.plot_overview import plot_simple_overview

plot_simple_overview(result_equal_area.data["dataset1"]);

In [ ]:
from pyglotaran_extras.io import setup_case_study

from glotaran.io import save_result

base_results_folder, _ = setup_case_study(output_folder_name="pyglotaran_examples_results")
results_folder = base_results_folder / "with_penalties_first_run"
save_result(result_equal_area, results_folder / "result.yml", allow_overwrite=True)

In [ ]:
from glotaran.io import save_dataset

for data_path in results_folder.rglob("*.nc"):
    save_dataset(load_dataset(data_path), results_folder / data_path.name, allow_overwrite=True)
    data_path.unlink()